In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from transformers import EncoderDecoderModel, AutoTokenizer
from typing import *
import tensorflow as tf
from transformers import TFEncoderDecoderModel, AutoTokenizer

import os
import urllib.request
from tqdm import tqdm
from copy import deepcopy

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


In [ ]:

class DownloadProgressBar(tqdm):
  def update_to(self, b=1, bsize=1, tsize=None):
    if tsize is not None:
      self.total = tsize
    self.update(b*bsize - self.n)

def download_url(url, output_path):
  with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=url.split('/')[-1]) as t:
    urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

def download_data(data_path, url_path, suffix):
  if not os.path.exists(data_path):
    os.makedirs(data_path)

  data_path = os.path.join(data_path,f'{suffix}.json')

  if not os.path.exists(data_path):
    print(f"Downloading CoQA {suffix} data split... (it may take a while)")
    download_url(url=url_path, output_path=data_path)
    print("Download Completed!")


In [ ]:
#Train Data
train_url = "https://nlp.stanford.edu/data/coqa/coqa-train-v1.0.json"
download_data(data_path='coqa',url_path=train_url, suffix='train')

#Test Data
test_url = "https://nlp.stanford.edu/data/coqa/coqa-dev-v1.0.json"
download_data(data_path='coqa', url_path = test_url, suffix='test')

In [ ]:
train_data = json.load((open('/content/coqa/train.json')))
qas = pd.json_normalize(train_data['data'], ['questions'], ['source', 'id', 'story'])
ans = pd.json_normalize(train_data['data'], ['answers'],['id'])
train_df = pd.merge(qas,ans, left_on=['id','turn_id'], right_on=['id','turn_id'])
train_df.loc[10:30,['turn_id','input_text_x', 'input_text_y', 'span_text']]

,turn_id,input_text_x,input_text_y,span_text
10,11,when were the Secret Archives moved from the r...,at the beginning of the 17th century;,atican Secret Archives were separated from the...
11,12,how many items are in this secret collection?,"150,000",Vatican Secret Archives were separated from t...
12,13,Can anyone use this library?,anyone who can document their qualifications a...,The Vatican Library is open to anyone who can...
13,14,what must be requested to view?,unknown,unknown
14,15,what must be requested in person or by mail?,Photocopies,Photocopies for private study of pages from bo...
15,16,of what books?,only books published between 1801 and 1990,hotocopies for private study of pages from boo...
16,17,What is the Vat the library of?,the Holy See,"simply the Vat, is the library of the Holy See,"
17,18,How many books survived the Pre Lateran period?,a handful of volumes,"Pre-Lateran period, comprising the initial day..."
18,19,what is the point of the project started in 2014?,digitising manuscripts,Vatican Library began an initial four-year pro...
19,20,what will this allow?,them to be viewed online.,"manuscripts, to be made available online."


In [ ]:
train_df['q_first_word']=train_df['input_text_x'].str.lower().str.extract(r'(\w+)')
train_df['q_first_two_words']=train_df['input_text_x'].str.lower().str.extract(r'^((?:\S+\s+){1}\S+).*')
train_df.groupby('q_first_word').count().sort_values(by='input_text_x',ascending=False).head(30)

,input_text_x,turn_id,bad_turn_x,source,id,story,span_start,span_end,span_text,input_text_y,bad_turn_y,q_first_two_words
q_first_word,,,,,,,,,,,,
what,32092,32092,114,32092,32092,32092,32092,32092,32092,32092,611,31711
who,15684,15684,45,15684,15684,15684,15684,15684,15684,15684,301,15075
how,10946,10946,37,10946,10946,10946,10946,10946,10946,10946,224,10662
did,7381,7381,19,7381,7381,7381,7381,7381,7381,7381,137,7381
where,7214,7214,21,7214,7214,7214,7214,7214,7214,7214,121,6305
was,5121,5121,30,5121,5121,5121,5121,5121,5121,5121,121,5121
when,4530,4530,10,4530,4530,4530,4530,4530,4530,4530,83,3614
is,3431,3431,16,3431,3431,3431,3431,3431,3431,3431,76,3431
why,2921,2921,13,2921,2921,2921,2921,2921,2921,2921,65,1885


In [ ]:
train_df.groupby('q_first_two_words').count().sort_values(by='input_text_x',ascending=False).head(30)

,input_text_x,turn_id,bad_turn_x,source,id,story,span_start,span_end,span_text,input_text_y,bad_turn_y,q_first_word
q_first_two_words,,,,,,,,,,,,
what did,5622,5622,27,5622,5622,5622,5622,5622,5622,5622,97,5622
what was,5079,5079,10,5079,5079,5079,5079,5079,5079,5079,100,5079
what is,4800,4800,15,4800,4800,4800,4800,4800,4800,4800,101,4800
how many,3692,3692,12,3692,3692,3692,3692,3692,3692,3692,108,3692
who was,3390,3390,9,3390,3390,3390,3390,3390,3390,3390,74,3390
who is,2409,2409,11,2409,2409,2409,2409,2409,2409,2409,29,2409
did he,2366,2366,5,2366,2366,2366,2366,2366,2366,2366,40,2366
where did,1988,1988,3,1988,1988,1988,1988,1988,1988,1988,40,1988
when did,1810,1810,5,1810,1810,1810,1810,1810,1810,1810,38,1810


In [ ]:
train_df = train_df.loc[train_df['input_text_y']!='unknown']
train_df.loc[10:30,['turn_id','input_text_x', 'input_text_y', 'span_text']]

,turn_id,input_text_x,input_text_y,span_text
10,11,when were the Secret Archives moved from the r...,at the beginning of the 17th century;,atican Secret Archives were separated from the...
11,12,how many items are in this secret collection?,"150,000",Vatican Secret Archives were separated from t...
12,13,Can anyone use this library?,anyone who can document their qualifications a...,The Vatican Library is open to anyone who can...
14,15,what must be requested in person or by mail?,Photocopies,Photocopies for private study of pages from bo...
15,16,of what books?,only books published between 1801 and 1990,hotocopies for private study of pages from boo...
16,17,What is the Vat the library of?,the Holy See,"simply the Vat, is the library of the Holy See,"
17,18,How many books survived the Pre Lateran period?,a handful of volumes,"Pre-Lateran period, comprising the initial day..."
18,19,what is the point of the project started in 2014?,digitising manuscripts,Vatican Library began an initial four-year pro...
19,20,what will this allow?,them to be viewed online.,"manuscripts, to be made available online."
20,1,Where was the Auction held?,Hard Rock Cafe,Hard Rock Cafe in New York's Times Square


In [ ]:
test_data = json.load((open('/content/coqa/test.json')))
qas = pd.json_normalize(test_data['data'], ['questions'], ['source', 'id', 'story'])
ans = pd.json_normalize(test_data['data'], ['answers'],['id'])
test_df = pd.merge(qas,ans, left_on=['id','turn_id'], right_on=['id','turn_id'])
test_df.loc[10:30,['turn_id','input_text_x', 'input_text_y', 'span_text']]

,turn_id,input_text_x,input_text_y,span_text
10,11,What did the other cats do when Cotton emerged...,licked her face,Her sisters licked her face
11,12,Did they want Cotton to change the color of he...,no,We would never want you to be any other way
12,1,what was the name of the fish,Asta.,Asta.
13,2,What looked like a birds belly,a bottle,a bottle
14,3,who said that,Asta.,"""It looks like a bird's belly,"" said Asta."
15,4,Was Sharkie a friend?,Yes,Asta's friend Sharkie
16,5,did they get the bottle?,Yes,So they caught the bottle
17,6,What was in it,a note,It was a note.
18,7,Did a little boy write the note,No,This note is from a little girl
19,8,Who could read the note,Asta's papa,Asta's papa read the note


In [ ]:
test_df = test_df.loc[test_df['input_text_y']!='unknown']
test_df.loc[10:30,['turn_id','input_text_x','input_text_y','span_text']]

,turn_id,input_text_x,input_text_y,span_text
10,11,What did the other cats do when Cotton emerged...,licked her face,Her sisters licked her face
11,12,Did they want Cotton to change the color of he...,no,We would never want you to be any other way
12,1,what was the name of the fish,Asta.,Asta.
13,2,What looked like a birds belly,a bottle,a bottle
14,3,who said that,Asta.,"""It looks like a bird's belly,"" said Asta."
15,4,Was Sharkie a friend?,Yes,Asta's friend Sharkie
16,5,did they get the bottle?,Yes,So they caught the bottle
17,6,What was in it,a note,It was a note.
18,7,Did a little boy write the note,No,This note is from a little girl
19,8,Who could read the note,Asta's papa,Asta's papa read the note


In [ ]:
train, val = train_test_split(train_df, test_size=0.2, random_state=42)
train.head()

,input_text_x,turn_id,bad_turn_x,source,id,story,span_start,span_end,span_text,input_text_y,bad_turn_y,q_first_word,q_first_two_words
54860,So how did they get to 28?,20,NaN,race,39dd6s19jpbtyxnmal6qgea8wr2ze3,Where did that number come from? Eleven and Tw...,1639,1740,he took one day from each of the 30-day months...,he took one day from each of the 30-day months...,NaN,so,so how
69607,How much was the package in value?,9,NaN,cnn,3ii4upycoj7fsz8vructj3gjsr7qdt,"Abidjan, Ivory Coast (CNN) -- The European Uni...",80,98,180 million euros,180 million euros,NaN,how,how much
94456,Did she think Adams was untrustworthy?,6,NaN,cnn,3wq3b2kge8gywyqusjv8nckbhrp1bi,"ATLANTA, Georgia (CNN) -- Michele Trobaugh reg...",426,462,She says she trusted him right away.,No,NaN,did,did she
94333,Who was he talking to?,3,NaN,gutenberg,3qapzx2qn4d41w5gd7yx8eyxhj320q,"CHAPTER V--""BLOODY AS THE HUNTER"" \n\nThe lads...",1208,1244,"""Ye but deride me,"" answered Matcham",Matcham,NaN,who,who was
47220,What does Pleistocene mean literally?,15,NaN,wikipedia,3nvc2eb65qzqj9xkpfnbjgx90ke3yk,"The Pleistocene (, often colloquially referred...",1410,1420,"""Most New""","""Most New.""",NaN,what,what does


In [ ]:
train = train[['story','input_text_x', 'input_text_y', 'span_text']]
val = val[['story','input_text_x', 'input_text_y', 'span_text']]
test_df = test_df[['story','input_text_x', 'input_text_y', 'span_text']]
display(train.head(),val.head(),test_df.head())

,story,input_text_x,input_text_y,span_text
54860,Where did that number come from? Eleven and Tw...,So how did they get to 28?,he took one day from each of the 30-day months...,he took one day from each of the 30-day months...
69607,"Abidjan, Ivory Coast (CNN) -- The European Uni...",How much was the package in value?,180 million euros,180 million euros
94456,"ATLANTA, Georgia (CNN) -- Michele Trobaugh reg...",Did she think Adams was untrustworthy?,No,She says she trusted him right away.
94333,"CHAPTER V--""BLOODY AS THE HUNTER"" \n\nThe lads...",Who was he talking to?,Matcham,"""Ye but deride me,"" answered Matcham"
47220,"The Pleistocene (, often colloquially referred...",What does Pleistocene mean literally?,"""Most New.""","""Most New"""


,story,input_text_x,input_text_y,span_text
90158,"Hong Kong, officially the Hong Kong Special Ad...",Are they involved with China in these groups?,no,such as the Asia-Pacific Economic Cooperation ...
90995,Volleyball has become a worldwide sport that i...,Did it remain popular for that age group?,Yes,popular with all age groups
88481,(CNN)The suspect behind the knife attack on th...,was he invited to the event?,no,wasn't on the list of those invited
6129,A Sudanese woman sentenced to die for refusing...,What about her mother's religion?,she was Ethiopian Orthodox,her mother was Ethiopian Orthodox
14367,Baronets are a rank in the British aristocracy...,What was the cost?,"£1,095","£1,095"


,story,input_text_x,input_text_y,span_text
0,"Once upon a time, in a barn near a farm house,...",What color was Cotton?,white,a little white kitten named Cotton
1,"Once upon a time, in a barn near a farm house,...",Where did she live?,in a barn,"in a barn near a farm house, there lived a lit..."
2,"Once upon a time, in a barn near a farm house,...",Did she live alone?,no,Cotton wasn't alone
3,"Once upon a time, in a barn near a farm house,...",Who did she live with?,with her mommy and 5 sisters,with her mommy and 5 other sisters
4,"Once upon a time, in a barn near a farm house,...",What color were her sisters?,orange and white,her sisters were all orange with beautiful whi...


#Defining the model and trainer for the pre-trained models

In [ ]:
class Trainer(object):
  """
  train_op -> use tf.GradentTape to compute loss
  batch_fit -> receives a batch and performs forward-backward passes
  """

  def __init__(self, keras_model):
    self.keras_model = keras_model
    self.optimizer = tf.keras.optimizers.Adam(learning_rate=5e-05)

  @tf.function
  def compute_loss(self, inputs):
    loss = self.keras_model(inputs=inputs)
    return tf.reduce_mean(loss)

  @tf.function
  def train_op(self, inputs):
    with tf.GradientTape() as tape:
      loss = self.compute_loss(inputs=inputs)
    
    grads = tape.gradient(loss, self.keras_model.trainable_variables)
    return loss, grads

  @tf.function
  def batch_fit(self, inputs):
    loss, grads = self.train_op(inputs = inputs)
    self.optimizer.apply_gradients(zip(grads, self.keras_model.trainable_variables))
    return losst

class Model(tf.keras.Model):
  #Custom keras model wrapping TFEncoderDecoderModel

  def __init__(self, model_name, **kwargs):
        super(Model, self).__init__(**kwargs)
        self.model_name = model_name

        # tie_encoder_decoder to share weights and half the number of parameters
        self.model = TFEncoderDecoderModel.from_encoder_decoder_pretrained(model_name, model_name,
                                                                           encoder_from_pt=True,
                                                                           decoder_from_pt=True,
                                                                           tie_encoder_decoder=True)

  def call(self, inputs, **kwargs):
      loss = self.model(input_ids=inputs['input_ids'],
                        attention_mask=inputs['input_attention_mask'],
                        decoder_input_ids=inputs['decoder_input_ids'],
                        decoder_attention_mask=inputs['labels_mask'],
                        labels=inputs['labels']).loss
      return loss

  def generate(self, **kwargs):
      return self.model.generate(decoder_start_token_id=self.model.config.decoder.pad_token_id,
                                  **kwargs)




In [ ]:
print(train.input_text_x)
questions = [i for i in train.input_text_x[:100]]
print(questions)
answers = [i for i in train.input_text_y[:100]]

54860                            So how did they get to 28?
69607                    How much was the package in value?
94456                Did she think Adams was untrustworthy?
94333                                Who was he talking to?
47220                 What does Pleistocene mean literally?
                                ...                        
55625                        Was Dick Rover a handsome man?
77830                                                 When?
105015                            About how many invasions?
874                           Who else was mentioned in it?
16003     Are they planning on hanging out together over...
Name: input_text_x, Length: 85820, dtype: object
['So how did they get to 28?', 'How much was the package in value?', 'Did she think Adams was untrustworthy?', 'Who was he talking to?', 'What does Pleistocene mean literally?', 'What was his real name?', "Were Bernstein's parents born in the US?", 'Did they eat pizza?', 'was there another 

In [ ]:
if __name__=='__main__':
  model_name = 'distilroberta-base'

  tokenizer = AutoTokenizer.from_pretrained(model_name, tie_encoder_decoder=True)
  model = Model(model_name = model_name)
  model.model.config.decoder_start_token_id = tokenizer.cls_token_id
  model.model.config_eos_token_id = tokenizer.sep_token_id
  model.model.config.pad_token_id = tokenizer.pad_token_id
  model.model.config.vocab_size = model.model.config.encoder.vocab_size

  trainer = Trainer(keras_model=model)
  
  input_values = tokenizer(questions, add_special_tokens=False, padding=True)
  input_ids, input_attention_mask = input_values['input_ids'], input_values['attention_mask']
  label_values = tokenizer(answers, padding=True)
  labels, labels_mask = label_values['input_ids'], label_values['attention_mask']

  max_length = 100

  masked_labels = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in labels]

  epochs = 20
  for epoch in tqdm(range(epochs)):
      batch = {'input_ids': tf.convert_to_tensor(input_ids, dtype=tf.int32),
                'input_attention_mask': tf.convert_to_tensor(input_attention_mask, dtype=tf.int32),
                'labels': tf.convert_to_tensor(masked_labels, dtype=tf.int32),
                'decoder_input_ids': tf.convert_to_tensor(deepcopy(labels), dtype=tf.int32),
                'labels_mask': tf.convert_to_tensor(labels_mask, dtype=tf.int32)
                }
      loss = trainer.batch_fit(inputs=batch)
      print(f'Epoch {epoch} -- Loss {loss}')

      # You can play with generation arguments to enforce
      #  beam search
      #  repetition penalty
      #  other sampling approaches
      generated = trainer.keras_model.generate(input_ids=tf.convert_to_tensor(input_ids, dtype=tf.int32),
                                                max_length=max_length,
                                                repetition_penalty=3.,
                                                min_length=4,
                                                no_repeat_ngram_size=3,
                                                early_stopping=True,
                                                num_beams=4
                                                )
      generated = tokenizer.batch_decode(generated, skip_special_tokens=True)
      print(f'Generated: {generated}')


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaModel.


{'input_ids': [[2847, 141, 222, 51, 120, 7, 971, 116, 1, 1, 1, 1, 1], [6179, 203, 21, 5, 3737, 11, 923, 116, 1, 1, 1, 1, 1], [20328, 79, 206, 5710, 21, 7587, 23705, 17328, 116, 1, 1, 1, 1], [12375, 21, 37, 1686, 7, 116, 1, 1, 1, 1, 1, 1, 1], [2264, 473, 18486, 661, 42416, 1266, 5909, 116, 1, 1, 1, 1, 1], [2264, 21, 39, 588, 766, 116, 1, 1, 1, 1, 1, 1, 1], [43822, 18211, 18, 1041, 2421, 11, 5, 382, 116, 1, 1, 1, 1], [20328, 51, 3529, 9366, 116, 1, 1, 1, 1, 1, 1, 1, 1], [7325, 89, 277, 169, 37, 5, 11054, 116, 1, 1, 1, 1, 1], [2409, 596, 21, 79, 1618, 116, 1, 1, 1, 1, 1, 1, 1], [20328, 10, 319, 9, 5450, 33, 1021, 38615, 116, 1, 1, 1, 1], [32112, 79, 10, 205, 23586, 116, 1, 1, 1, 1, 1, 1, 1], [2409, 277, 116, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [12196, 817, 24, 173, 116, 1, 1, 1, 1, 1, 1, 1, 1], [8275, 39, 1041, 33, 10, 2199, 116, 1, 1, 1, 1, 1, 1], [12375, 341, 7, 9450, 13, 1292, 13804, 116, 1, 1, 1, 1, 1], [35299, 47, 555, 10, 919, 71, 744, 116, 1, 1, 1, 1, 1], [12196, 222, 79, 109, 11, 69, 3

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/tensorflow/python/autograph/impl/api.py:371: FutureWarning: Version v4.17.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.17.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  return py_builtins.overload_of(f)(*args)


Epoch 0 -- Loss 9.889678001403809


  5%|▌         | 1/20 [05:42<1:48:30, 342.66s/it]

...

Generated: [' the of and to.,-" aTheAathe yearsthno "000 is it in for other from\'s areNo', ' the of. and to-," aTheaAthenoth000 years "No it\'sand is other fors from', ' the of and. to-," aTheaAthenoth years "000No it\'sand is for otherL from', ' the of. and to-,"The aaAtheno yearsth "000No isand it\'s otherL fors', ' the of. and to-," aTheaAthenoth " years000 itandNo is\'s forsL in', ' the of and. to-,"The aaAthenoth "000 yearsNoand\'s isLn other from for', ' the of. and to-," aTheaAthenoth000 years "No it isand\'s other for fromL', ' the of. and to-," aTheatheAnoth years "000and itNo is\'s for inn from', ' the of. to and"-, aTheaAthenoth " years000 it\'sandNo isLs for in', ' the of. and to-," aTheatheAnoth years "000No it\'sand is otherLs for', ' the of. and to-," aTheaAthe yearsnoth "000 is itNo for in\'sand are from', ' the of and. to-," aTheaAthenoth years000 " is it\'sandNo fors from other', ' the of. and to,"- aTheatheAthno years "000and is it for\'sNosL in', ' the of. and to-,

 95%|█████████▌| 19/20 [1:39:28<05:10, 310.60s/it]

Generated: [' the of and to.,"- aThetheno yearsAath "No000 is in from other itL for\'s', ' the of. and to"-,The atheanoA yearsth "No000and it\'sLn is from in', ' the of and." to-,The aanotheAth yearsNo "000andLn\'s it is from other', ' the of. and to"-,The athenoAa yearsthNo "000and isL\'sn other it from', ' the of. and to"-,The atheanoAth years "Noand000Ln it\'s is in from', ' the of. and to"-,Thethenoa aAth years "No000andnL\'sto from other is', ' the of. and to"-,Thea anotheAth yearsNo "000and itLn\'s is from other', ' the of. and to"-,The atheanoA yearsth "Noand000nL\'s it is from in', ' the of. and to"-,The aatheAnoth " yearsNoand000L\'sn it iss from', ' the of. and to"-,The anotheaAth yearsNo "000andL\'sn other it is from', ' the of. and to,"-The athenoaA yearsth "No000 isand it in\'sL from for', ' the of and." to-,The aathenoAth yearsNo "000and\'s it isLn from for', ' the of. and to"-,Thethea anoAth years "Noand000 itLn\'s is for from', ' the of. and to"-,The atheanoA yearsth "N

100%|██████████| 20/20 [1:44:44<00:00, 314.22s/it]

Generated: [' the of and. to",-noThethe years aAath "No000 fromL in other is itand their', ' the of. and to-",notheThe aaA yearsthNo "000andtoLn\'s from other it', ' the of and." to-,notheThea aA yearsNoth "and000Lnto\'s from other it', ' the of. and to"-,notheThe aAa yearsthNo "and000Lnto from\'s other is', ' the of and."- to,notheThea aA yearsth "Noand000Lnto from in\'s it', ' the of. and to-"no,theTheaA ath yearsNo "toand000nL other from\'s their', ' the of and." to-,notheTheaA ath yearsNo "000andtoLn from other\'s it', ' the of and." to-,notheThea aA yearsthNo "and000nLto from in\'s other', ' the of and." to-no,theTheaA ath years "NoandLton000\'s from in it', ' the of. and to-"no,theThe aaA yearsthNo "Landto000n other\'s from it', ' the of. and to"-,noThethe aaA yearsthNo "000and isLn it in from\'s', ' the of and." to-,notheThea aA yearsthNo "and000Ln from\'sto other it', ' the of and." to-,notheThea aA yearsth "NoandntoL000\'s from it in', ' the of and." to-,notheThe aaA yearsthNo